<a href="https://colab.research.google.com/github/hl5chen/ECE1512-2022F-ProjectRepo_YiyangShi_HsuanlingChen/blob/main/Project_B_Supp/DM/ECE1512_T2_DM_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab Notebooks/ECE1512_F22/ProjectB

/content/drive/MyDrive/Colab Notebooks/ECE1512_F22/ProjectB


In [ ]:
!pip install import-ipynb
import import_ipynb

import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
from sys import argv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 37.3 MB/s 


# Dataset Condensation with Distribution Matching



In [ ]:
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug


def main(args):

    #parser = argparse.ArgumentParser(description='Parameter Processing')
    #parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset')
    #parser.add_argument('--model', type=str, default='ConvNet', help='model')
    #parser.add_argument('--ipc', type=int, default=10, help='image(s) per class')
    #parser.add_argument('--eval_mode', type=str, default='S', help='eval_mode') # was SS # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
    #parser.add_argument('--num_exp', type=int, default=1, help='the number of experiments') # was 5
    #parser.add_argument('--num_eval', type=int, default=1, help='the number of evaluating randomly initialized models') # was 20
    #parser.add_argument('--epoch_eval_train', type=int, default=100, help='epochs to train a model with synthetic data') # was 1000 # it can be small for speeding up with little performance drop
    #parser.add_argument('--Iteration', type=int, default=2000, help='training iterations') #was 20000
    #parser.add_argument('--lr_img', type=float, default=1.0, help='learning rate for updating synthetic images')
    #parser.add_argument('--lr_net', type=float, default=0.01, help='learning rate for updating network parameters')
    #parser.add_argument('--batch_real', type=int, default=256, help='batch size for real data')
    #parser.add_argument('--batch_train', type=int, default=256, help='batch size for training networks')
    #parser.add_argument('--init', type=str, default='real', help='noise/real: initialize synthetic images from random noise or randomly sampled real images.')
    #parser.add_argument('--dsa_strategy', type=str, default='color_crop_cutout_flip_scale_rotate', help='differentiable Siamese augmentation strategy')
    #parser.add_argument('--data_path', type=str, default='DM_data_CIFAR10', help='dataset path')
    #parser.add_argument('--save_path', type=str, default='DM_result_CIFAR10', help='path to save results')
    #parser.add_argument('--dis_metric', type=str, default='ours', help='distance metric')

    #args = parser.parse_args()
    args.method = 'DM'
    args.outer_loop, args.inner_loop = get_loops(args.ipc)
    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    args.dsa_param = ParamDiffAug()
    args.dsa = False if args.dsa_strategy in ['none', 'None'] else True

    if not os.path.exists(args.data_path):
        os.mkdir(args.data_path)

    if not os.path.exists(args.save_path):
        os.mkdir(args.save_path)

    eval_it_pool = np.arange(0, args.Iteration+1, 2000).tolist() if args.eval_mode == 'S' or args.eval_mode == 'SS' else [args.Iteration] # The list of iterations when we evaluate models and record results.
    print('eval_it_pool: ', eval_it_pool)
    channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
    model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)


    accs_all_exps = dict() # record performances of all experiments
    for key in model_eval_pool:
        accs_all_exps[key] = []

    data_save = []


    for exp in range(args.num_exp):
        print('\n================== Exp %d ==================\n '%exp)
        print('Hyper-parameters: \n', args.__dict__)
        print('Evaluation model pool: ', model_eval_pool)

        ''' organize the real dataset '''
        images_all = []
        labels_all = []
        indices_class = [[] for c in range(num_classes)]

        images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
        labels_all = [dst_train[i][1] for i in range(len(dst_train))]
        for i, lab in enumerate(labels_all):
            indices_class[lab].append(i)
        images_all = torch.cat(images_all, dim=0).to(args.device)
        labels_all = torch.tensor(labels_all, dtype=torch.long, device=args.device)

        for c in range(num_classes):
            print('class c = %d: %d real images'%(c, len(indices_class[c])))

        def get_images(c, n): # get random n images from class c
            idx_shuffle = np.random.permutation(indices_class[c])[:n]
            return images_all[idx_shuffle]

        for ch in range(channel):
            print('real images channel %d, mean = %.4f, std = %.4f'%(ch, torch.mean(images_all[:, ch]), torch.std(images_all[:, ch])))

        ''' initialize the synthetic data '''
        image_syn = torch.randn(size=(num_classes*args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=args.device)
        label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]

        if args.init == 'real':
            print('initialize synthetic data from random real images')
            for c in range(num_classes):
                image_syn.data[c*args.ipc:(c+1)*args.ipc] = get_images(c, args.ipc).detach().data
        else:
            print('initialize synthetic data from random noise')


        ''' training '''
        optimizer_img = torch.optim.SGD([image_syn, ], lr=args.lr_img, momentum=0.5) # optimizer_img for synthetic data
        optimizer_img.zero_grad()
        print('%s training begins'%get_time())

        for it in range(args.Iteration+1):
            ''' Evaluate synthetic data '''
            if it in eval_it_pool:
                for model_eval in model_eval_pool:
                    print('-------------------------\nEvaluation\nmodel_train = %s, model_eval = %s, iteration = %d'%(args.model, model_eval, it))

                    print('DSA augmentation strategy: \n', args.dsa_strategy)
                    print('DSA augmentation parameters: \n', args.dsa_param.__dict__)

                    accs = []
                    for it_eval in range(args.num_eval):
                        net_eval = get_network(model_eval, channel, num_classes, im_size).to(args.device) # get a random model
                        image_syn_eval, label_syn_eval = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach()) # avoid any unaware modification
                        _, acc_train, acc_test = evaluate_synset(it_eval, net_eval, image_syn_eval, label_syn_eval, testloader, args)
                        accs.append(acc_test)
                    print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

                    if it == args.Iteration: # record the final results
                        accs_all_exps[model_eval] += accs

                ''' visualize and save '''
                save_name = os.path.join(args.save_path, 'vis_%s_%s_%s_%dipc_exp%d_iter%d.png'%(args.method, args.dataset, args.model, args.ipc, exp, it))
                image_syn_vis = copy.deepcopy(image_syn.detach().cpu())
                for ch in range(channel):
                    image_syn_vis[:, ch] = image_syn_vis[:, ch]  * std[ch] + mean[ch]
                image_syn_vis[image_syn_vis<0] = 0.0
                image_syn_vis[image_syn_vis>1] = 1.0
                save_image(image_syn_vis, save_name, nrow=args.ipc) # Trying normalize = True/False may get better visual effects.



            ''' Train synthetic data '''
            net = get_network(args.model, channel, num_classes, im_size).to(args.device) # get a random model
            net.train()
            for param in list(net.parameters()):
                param.requires_grad = False

            embed = net.module.embed if torch.cuda.device_count() > 1 else net.embed # for GPU parallel

            loss_avg = 0

            ''' update synthetic data '''
            if 'BN' not in args.model: # for ConvNet
                loss = torch.tensor(0.0).to(args.device)
                for c in range(num_classes):
                    img_real = get_images(c, args.batch_real)
                    img_syn = image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))

                    if args.dsa:
                        seed = int(time.time() * 1000) % 100000
                        img_real = DiffAugment(img_real, args.dsa_strategy, seed=seed, param=args.dsa_param)
                        img_syn = DiffAugment(img_syn, args.dsa_strategy, seed=seed, param=args.dsa_param)

                    output_real = embed(img_real).detach()
                    output_syn = embed(img_syn)

                    loss += torch.sum((torch.mean(output_real, dim=0) - torch.mean(output_syn, dim=0))**2)

            else: # for ConvNetBN
                images_real_all = []
                images_syn_all = []
                loss = torch.tensor(0.0).to(args.device)
                for c in range(num_classes):
                    img_real = get_images(c, args.batch_real)
                    img_syn = image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))

                    if args.dsa:
                        seed = int(time.time() * 1000) % 100000
                        img_real = DiffAugment(img_real, args.dsa_strategy, seed=seed, param=args.dsa_param)
                        img_syn = DiffAugment(img_syn, args.dsa_strategy, seed=seed, param=args.dsa_param)

                    images_real_all.append(img_real)
                    images_syn_all.append(img_syn)

                images_real_all = torch.cat(images_real_all, dim=0)
                images_syn_all = torch.cat(images_syn_all, dim=0)

                output_real = embed(images_real_all).detach()
                output_syn = embed(images_syn_all)

                loss += torch.sum((torch.mean(output_real.reshape(num_classes, args.batch_real, -1), dim=1) - torch.mean(output_syn.reshape(num_classes, args.ipc, -1), dim=1))**2)



            optimizer_img.zero_grad()
            loss.backward()
            optimizer_img.step()
            loss_avg += loss.item()


            loss_avg /= (num_classes)

            if it%10 == 0:
                print('%s iter = %05d, loss = %.4f' % (get_time(), it, loss_avg))

            if it == args.Iteration: # only record the final results
                data_save.append([copy.deepcopy(image_syn.detach().cpu()), copy.deepcopy(label_syn.detach().cpu())])
                torch.save({'data': data_save, 'accs_all_exps': accs_all_exps, }, os.path.join(args.save_path, 'res_%s_%s_%s_%dipc.pt'%(args.method, args.dataset, args.model, args.ipc)))


    print('\n==================== Final Results ====================\n')
    for key in model_eval_pool:
        accs = accs_all_exps[key]
        print('Run %d experiments, train on %s, evaluate %d random %s, mean  = %.2f%%  std = %.2f%%'%(args.num_exp, args.model, len(accs), key, np.mean(accs)*100, np.std(accs)*100))



## MNIST

In [ ]:
class arguments():
    def __init__(self,): 
        self.method = 'DM'
        self.dataset = 'MNIST'
        self.model = 'ConvNet'
        self.ipc = 10
        self.eval_mode = 'S' # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
        self.num_exp = 1
        self.num_eval = 1
        self.epoch_eval_train = 100
        self.Iteration = 2000
        self.lr_img = 1.0
        self.lr_net = 0.01
        self.batch_real = 256
        self.batch_train = 256
        self.init = 'real'  #'noise/real: initialize synthetic images from random noise or randomly sampled real images.'
        self.dsa_strategy = 'color_crop_cutout_flip_scale_rotate'
        self.data_path = 'MNISTdata_real'
        self.save_path = 'MNISTresult_real'
        self.dis_metric = 'ours'
        self.outer_loop = 10
        self.inner_loop = 50
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dsa_param = ParamDiffAug()
        self.dsa = 'False'
        
args = arguments()
main(args)

eval_it_pool:  [0, 2000]


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting CIFAR10data/MNIST/raw/train-images-idx3-ubyte.gz to CIFAR10data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting CIFAR10data/MNIST/raw/train-labels-idx1-ubyte.gz to CIFAR10data/MNIST/raw



...

Extracting CIFAR10data/MNIST/raw/t10k-images-idx3-ubyte.gz to CIFAR10data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting CIFAR10data/MNIST/raw/t10k-labels-idx1-ubyte.gz to CIFAR10data/MNIST/raw


================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DM', 'dataset': 'MNIST', 'model': 'ConvNet', 'ipc': 10, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 1, 'epoch_eval_train': 100, 'Iteration': 2000, 'lr_img': 1.0, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'real', 'dsa_strategy': 'color_crop_cutout_flip_scale_rotate', 'data_path': 'CIFAR10data', 'save_path': 'CIFAR10result', 'dis_metric': 'ours', 'outer_loop': 10, 'inner_loop': 50, 'device': 'cuda', 'dsa_param': <utils.ParamDiffAug object at 0x7f2bfbe48a30>, 'dsa': True}
Evaluation model pool:  ['ConvNet']
class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5949 real images
real 

<ipython-input-11-d63b2f320503>:88: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


[2022-12-09 19:58:49] Evaluate_00: epoch = 0100 train time = 1 s train loss = 0.033364 train acc = 1.0000, test acc = 0.9026
Evaluate 1 random ConvNet, mean = 0.9026 std = 0.0000
-------------------------
[2022-12-09 19:58:49] iter = 00000, loss = 8.5273
[2022-12-09 19:58:50] iter = 00010, loss = 4.0605
[2022-12-09 19:58:52] iter = 00020, loss = 3.4108
[2022-12-09 19:58:54] iter = 00030, loss = 3.2923
[2022-12-09 19:58:55] iter = 00040, loss = 2.4211
[2022-12-09 19:58:57] iter = 00050, loss = 2.2930
[2022-12-09 19:58:58] iter = 00060, loss = 2.1447
[2022-12-09 19:59:00] iter = 00070, loss = 2.1699
[2022-12-09 19:59:02] iter = 00080, loss = 2.0039
[2022-12-09 19:59:03] iter = 00090, loss = 1.8370
[2022-12-09 19:59:05] iter = 00100, loss = 1.8855
[2022-12-09 19:59:06] iter = 00110, loss = 1.6227
[2022-12-09 19:59:08] iter = 00120, loss = 1.6509
[2022-12-09 19:59:10] iter = 00130, loss = 1.5793
[2022-12-09 19:59:11] iter = 00140, loss = 1.7164
[2022-12-09 19:59:13] iter = 00150, loss = 1.

In [ ]:
class arguments():
    def __init__(self,): 
        self.method = 'DM'
        self.dataset = 'MNIST'
        self.model = 'ConvNet'
        self.ipc = 10
        self.eval_mode = 'S' # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
        self.num_exp = 1
        self.num_eval = 1
        self.epoch_eval_train = 100
        self.Iteration = 2000
        self.lr_img = 1.0
        self.lr_net = 0.01
        self.batch_real = 256
        self.batch_train = 256
        self.init = 'noise'  #'noise/real: initialize synthetic images from random noise or randomly sampled real images.'
        self.dsa_strategy = 'color_crop_cutout_flip_scale_rotate'
        self.data_path = 'MNIST_data_noise'
        self.save_path = 'MNIST_result_noise'
        self.dis_metric = 'ours'
        self.outer_loop = 10
        self.inner_loop = 50
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dsa_param = ParamDiffAug()
        self.dsa = 'False'
        
args = arguments()
main(args)

eval_it_pool:  [0, 2000]


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data_noise/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data_noise/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data_noise/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data_noise/MNIST/raw



...

Extracting MNIST_data_noise/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data_noise/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data_noise/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data_noise/MNIST/raw


================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DM', 'dataset': 'MNIST', 'model': 'ConvNet', 'ipc': 10, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 1, 'epoch_eval_train': 100, 'Iteration': 2000, 'lr_img': 1.0, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'noise', 'dsa_strategy': 'color_crop_cutout_flip_scale_rotate', 'data_path': 'MNIST_data_noise', 'save_path': 'MNIST_result_noise', 'dis_metric': 'ours', 'outer_loop': 10, 'inner_loop': 50, 'device': 'cuda', 'dsa_param': <utils.ParamDiffAug object at 0x7f15df0203d0>, 'dsa': True}
Evaluation model pool:  ['ConvNet']
class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5

<ipython-input-4-d63b2f320503>:88: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]
<ipython-input-4-d63b2f320503>:88: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to p

[2022-12-11 23:08:47] Evaluate_00: epoch = 0100 train time = 7 s train loss = 0.173053 train acc = 1.0000, test acc = 0.1244
Evaluate 1 random ConvNet, mean = 0.1244 std = 0.0000
-------------------------
[2022-12-11 23:08:48] iter = 00000, loss = 199.4511
[2022-12-11 23:08:49] iter = 00010, loss = 32.5887
[2022-12-11 23:08:51] iter = 00020, loss = 19.2363
[2022-12-11 23:08:53] iter = 00030, loss = 14.7205
[2022-12-11 23:08:54] iter = 00040, loss = 10.6274
[2022-12-11 23:08:56] iter = 00050, loss = 8.5590
[2022-12-11 23:08:58] iter = 00060, loss = 7.6868
[2022-12-11 23:08:59] iter = 00070, loss = 7.4576
[2022-12-11 23:09:01] iter = 00080, loss = 6.1036
[2022-12-11 23:09:02] iter = 00090, loss = 5.7607
[2022-12-11 23:09:04] iter = 00100, loss = 5.2618
[2022-12-11 23:09:05] iter = 00110, loss = 4.8844
[2022-12-11 23:09:07] iter = 00120, loss = 4.1259
[2022-12-11 23:09:09] iter = 00130, loss = 3.7584
[2022-12-11 23:09:10] iter = 00140, loss = 3.1793
[2022-12-11 23:09:12] iter = 00150, los

## CIFAR10

In [8]:
class arguments():
    def __init__(self,): 
        self.method = 'DM'
        self.dataset = 'CIFAR10'
        self.model = 'ConvNet'
        self.ipc = 10 #'image(s) per class'
        self.eval_mode = 'S' # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
        self.num_exp = 1 # number of experiments
        self.num_eval = 1 # number of evaluating randomly initialized models
        self.epoch_eval_train = 100
        self.Iteration = 2000
        self.lr_img = 1.0
        self.lr_net = 0.01
        self.batch_real = 256
        self.batch_train = 256
        self.init = 'real' #'noise/real: initialize synthetic images from random noise or randomly sampled real images.'
        self.dsa_strategy = 'color_crop_cutout_flip_scale_rotate'
        self.data_path = 'CIFAR10data_real'
        self.save_path = 'CIFAR10result_real'
        self.dis_metric = 'ours'
        self.outer_loop = 10
        self.inner_loop = 50
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dsa_param = ParamDiffAug()
        self.dsa = 'False'
        
args = arguments()
main(args)

eval_it_pool:  [0, 2000]


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting CIFAR10data_real/cifar-10-python.tar.gz to CIFAR10data_real
Files already downloaded and verified

================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DM', 'dataset': 'CIFAR10', 'model': 'ConvNet', 'ipc': 10, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 1, 'epoch_eval_train': 100, 'Iteration': 2000, 'lr_img': 1.0, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'real', 'dsa_strategy': 'color_crop_cutout_flip_scale_rotate', 'data_path': 'CIFAR10data_real', 'save_path': 'CIFAR10result_real', 'dis_metric': 'ours', 'outer_loop': 10, 'inner_loop': 50, 'device': 'cuda', 'dsa_param': <utils.ParamDiffAug object at 0x7f15561eb160>, 'dsa': True}
Evaluation model pool:  ['ConvNet']
class c = 0: 5000 real images
class c = 1: 5000 real images
class c = 2: 5000 real images
class c = 3: 5000 real images
class c = 4: 5000 real images
class c = 5: 5000 real images
class c = 6: 5000 real images
class c = 7: 5000 real images
class c = 8: 5000 real images

<ipython-input-4-d63b2f320503>:88: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


[2022-12-11 23:24:11] Evaluate_00: epoch = 0100 train time = 1 s train loss = 0.280810 train acc = 0.9200, test acc = 0.2904
Evaluate 1 random ConvNet, mean = 0.2904 std = 0.0000
-------------------------
[2022-12-11 23:24:12] iter = 00000, loss = 28.3654
[2022-12-11 23:24:13] iter = 00010, loss = 17.1228
[2022-12-11 23:24:15] iter = 00020, loss = 13.6984
[2022-12-11 23:24:16] iter = 00030, loss = 12.0306
[2022-12-11 23:24:18] iter = 00040, loss = 10.0813
[2022-12-11 23:24:19] iter = 00050, loss = 9.9213
[2022-12-11 23:24:21] iter = 00060, loss = 9.0263
[2022-12-11 23:24:23] iter = 00070, loss = 8.5477
[2022-12-11 23:24:24] iter = 00080, loss = 8.4705
[2022-12-11 23:24:26] iter = 00090, loss = 8.1531
[2022-12-11 23:24:27] iter = 00100, loss = 7.6011
[2022-12-11 23:24:29] iter = 00110, loss = 8.0050
[2022-12-11 23:24:30] iter = 00120, loss = 7.6195
[2022-12-11 23:24:32] iter = 00130, loss = 7.2166
[2022-12-11 23:24:34] iter = 00140, loss = 7.1989
[2022-12-11 23:24:35] iter = 00150, loss

In [9]:
class arguments():
    def __init__(self,): 
        self.method = 'DM'
        self.dataset = 'CIFAR10'
        self.model = 'ConvNet'
        self.ipc = 10 #'image(s) per class'
        self.eval_mode = 'S' # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
        self.num_exp = 1 # number of experiments
        self.num_eval = 1 # number of evaluating randomly initialized models
        self.epoch_eval_train = 100
        self.Iteration = 2000
        self.lr_img = 1.0
        self.lr_net = 0.01
        self.batch_real = 256
        self.batch_train = 256
        self.init = 'noise' #'noise/real: initialize synthetic images from random noise or randomly sampled real images.'
        self.dsa_strategy = 'color_crop_cutout_flip_scale_rotate'
        self.data_path = 'CIFAR10data_noise'
        self.save_path = 'CIFAR10result_noise'
        self.dis_metric = 'ours'
        self.outer_loop = 10
        self.inner_loop = 50
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dsa_param = ParamDiffAug()
        self.dsa = 'False'
        
args = arguments()
main(args)

eval_it_pool:  [0, 2000]
Files already downloaded and verified
Files already downloaded and verified

================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DM', 'dataset': 'CIFAR10', 'model': 'ConvNet', 'ipc': 10, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 1, 'epoch_eval_train': 100, 'Iteration': 2000, 'lr_img': 1.0, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'noise', 'dsa_strategy': 'color_crop_cutout_flip_scale_rotate', 'data_path': 'CIFAR10data_noise', 'save_path': 'CIFAR10result_noise', 'dis_metric': 'ours', 'outer_loop': 10, 'inner_loop': 50, 'device': 'cuda', 'dsa_param': <utils.ParamDiffAug object at 0x7f15df000550>, 'dsa': True}
Evaluation model pool:  ['ConvNet']
class c = 0: 5000 real images
class c = 1: 5000 real images
class c = 2: 5000 real images
class c = 3: 5000 real images
class c = 4: 5000 real images
class c = 5: 5000 real images
class c = 6: 5000 real images
class c = 7: 5000 real images
class c = 8: 5000 real images
clas

<ipython-input-4-d63b2f320503>:88: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]


[2022-12-11 23:32:37] Evaluate_00: epoch = 0100 train time = 1 s train loss = 0.684696 train acc = 0.8300, test acc = 0.1128
Evaluate 1 random ConvNet, mean = 0.1128 std = 0.0000
-------------------------
[2022-12-11 23:32:37] iter = 00000, loss = 62.0588
[2022-12-11 23:32:39] iter = 00010, loss = 29.6407
[2022-12-11 23:32:41] iter = 00020, loss = 26.1799
[2022-12-11 23:32:42] iter = 00030, loss = 34.7117
[2022-12-11 23:32:44] iter = 00040, loss = 22.3644
[2022-12-11 23:32:45] iter = 00050, loss = 22.1976
[2022-12-11 23:32:47] iter = 00060, loss = 21.8973
[2022-12-11 23:32:49] iter = 00070, loss = 30.6474
[2022-12-11 23:32:50] iter = 00080, loss = 21.8596
[2022-12-11 23:32:52] iter = 00090, loss = 22.3538
[2022-12-11 23:32:54] iter = 00100, loss = 21.7807
[2022-12-11 23:32:56] iter = 00110, loss = 15.9263
[2022-12-11 23:32:58] iter = 00120, loss = 11.5795
[2022-12-11 23:33:01] iter = 00130, loss = 19.2174
[2022-12-11 23:33:02] iter = 00140, loss = 15.8816
[2022-12-11 23:33:04] iter = 0